In [1]:
!nvidia-smi

Wed May 13 13:43:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import sys
import numpy as np
np.random.seed(0)
import keras
from keras.layers import Input, Dense, Lambda, Concatenate, Layer
from keras.models import Model
from keras import backend as K
from keras import initializers
from keras.engine import InputSpec
from keras.datasets import mnist

In [3]:
class ConcreteDropout(Layer):
    """This wrapper allows to learn the dropout probability for any given input Dense layer.
    ```python
        # as the first layer in a model
        model = Sequential()
        model.add(ConcreteDropout(Dense(8), input_shape=(16)))
        # now model.output_shape == (None, 8)
        # subsequent layers: no need for input_shape
        model.add(ConcreteDropout(Dense(32)))
        # now model.output_shape == (None, 32)
    ```
    `ConcreteDropout` can be used with arbitrary layers which have 2D
    kernels, not just `Dense`. However, Conv2D layers require different
    weighing of the regulariser (use SpatialConcreteDropout instead).
    # Arguments
        layer: a layer instance.
        weight_regularizer:
            A positive number which satisfies
                $weight_regularizer = l**2 / (\tau * N)$
            with prior lengthscale l, model precision $\tau$ (inverse observation noise),
            and N the number of instances in the dataset.
            Note that kernel_regularizer is not needed.
        dropout_regularizer:
            A positive number which satisfies
                $dropout_regularizer = 2 / (\tau * N)$
            with model precision $\tau$ (inverse observation noise) and N the number of
            instances in the dataset.
            Note the relation between dropout_regularizer and weight_regularizer:
                $weight_regularizer / dropout_regularizer = l**2 / 2$
            with prior lengthscale l. Note also that the factor of two should be
            ignored for cross-entropy loss, and used only for the eculedian loss.
    """

    def __init__(self, layer, weight_regularizer=1e-6, dropout_regularizer=1e-5,
                 init_min=0.1, init_max=0.1, is_mc_dropout=True, **kwargs):
        assert 'kernel_regularizer' not in kwargs
        super(ConcreteDropout, self).__init__(**kwargs)
        self.weight_regularizer = weight_regularizer
        self.dropout_regularizer = dropout_regularizer
        self.is_mc_dropout = is_mc_dropout
        self.supports_masking = True
        self.p_logit = None
        self.p = None
        self.init_min = np.log(init_min) - np.log(1. - init_min)
        self.init_max = np.log(init_max) - np.log(1. - init_max)
        self.layer = layer

    def build(self, input_shape=None):
        self.input_spec = InputSpec(shape=input_shape)
        if not self.layer.built:
            self.layer.build(input_shape)
            self.layer.built = True
        super(ConcreteDropout, self).build(input_shape=None)  # this is very weird.. we must call super before we add new losses

        # initialise p
        self.p_logit = self.layer.add_weight(name='p_logit',
                                            shape=(1,),
                                            initializer=initializers.RandomUniform(self.init_min, self.init_max),
                                            trainable=True)
        self.p = K.sigmoid(self.p_logit[0])

        # initialise regulariser / prior KL term
        assert len(input_shape) == 2, 'this wrapper only supports Dense layers'
        input_dim = np.prod(input_shape[-1])  # we drop only last dim
        weight = self.layer.kernel
        kernel_regularizer = self.weight_regularizer * K.sum(K.square(weight)) / (1. - self.p)
        dropout_regularizer = self.p * K.log(self.p)
        dropout_regularizer += (1. - self.p) * K.log(1. - self.p)
        dropout_regularizer *= self.dropout_regularizer * input_dim
        regularizer = K.sum(kernel_regularizer + dropout_regularizer)
        self.layer.add_loss(regularizer) # Eq 3 of the paper.

    def compute_output_shape(self, input_shape):
        return self.layer.compute_output_shape(input_shape)

    def concrete_dropout(self, x):
        '''
        Concrete dropout - used at training time (gradients can be propagated)
        :param x: input
        :return:  approx. dropped out input
        '''
        eps = K.cast_to_floatx(K.epsilon())
        temp = 0.1

        unif_noise = K.random_uniform(shape=K.shape(x))
        drop_prob = (
            K.log(self.p + eps)
            - K.log(1. - self.p + eps)
            + K.log(unif_noise + eps)
            - K.log(1. - unif_noise + eps)
        )
        drop_prob = K.sigmoid(drop_prob / temp)
        random_tensor = 1. - drop_prob

        retain_prob = 1. - self.p
        x *= random_tensor
        x /= retain_prob
        return x

    def call(self, inputs, training=None):
        if self.is_mc_dropout:
            return self.layer.call(self.concrete_dropout(inputs))
        else:
            def relaxed_dropped_inputs():
                return self.layer.call(self.concrete_dropout(inputs))
            return K.in_train_phase(relaxed_dropped_inputs,
                                    self.layer.call(inputs),
                                    training=training)

Using TensorFlow backend.


In [0]:
Q = 1 # dimension of data
D = 1 # One mean and one var
K_test = 20 # Number of MC samples
nb_reps = 3
batch_size = 20
l = 1e-4 # length scale
Q = 28*28
nb_features = 512
num_classes = 10
batch_size = 64

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
print('x_train shape:', X_train.shape)

11493376/11490434 [==============================] - 2s 0us/step
x_train shape: (60000, 784)


In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
def fit_model_1(nb_epoch, X, Y):
    if K.backend() == 'tensorflow':
        K.clear_session()
    N = X.shape[0]
    wd = l**2. / N
    dd = 2. / N
    print(N,wd,dd)
    inp = Input(shape=(Q,))
    x = inp
    x = ConcreteDropout(Dense(nb_features, activation='relu'), weight_regularizer=wd, dropout_regularizer=dd)(x)
    x = ConcreteDropout(Dense(nb_features, activation='relu'), weight_regularizer=wd, dropout_regularizer=dd)(x)
    x = ConcreteDropout(Dense(nb_features, activation='relu'), weight_regularizer=wd, dropout_regularizer=dd)(x)
    out = ConcreteDropout(Dense(num_classes, activation='softmax'), weight_regularizer=wd, dropout_regularizer=dd)(x)
    # out = ConcreteDropout(Dense(), weight_regularizer=wd, dropout_regularizer=dd)(x)
    # log_var = ConcreteDropout(Dense(D), weight_regularizer=wd, dropout_regularizer=dd)(x)
    # out = Concatenate()([mean, log_var])
    # out = merge([mean, log_var], mode='concat')
    model = Model(inp, out)
    
    # def heteroscedastic_loss(true, pred):
    #     mean = pred[:, :D]
    #     log_var = pred[:, D:]
    #     precision = K.exp(-log_var)
    #     return K.sum(precision * (true - mean)**2. + log_var, -1)
    
    model.compile(optimizer='adam', loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
    assert len(model.layers[1].trainable_weights) == 3  # kernel, bias, and dropout prob
    assert len(model.losses) == 4  # a loss for each Concrete Dropout layer
    hist = model.fit(X, Y, epochs=nb_epoch, batch_size=batch_size, verbose=1,validation_split=0.2)
    loss = hist.history['loss'][-1]
    return model, -0.5 * loss  # return ELBO up to const.

In [8]:
nb_epoch = 20
model, ELBO = fit_model_1(nb_epoch,X_train,y_train)
MC_samples = np.array([model.predict(X_test) for _ in range(K_test)])
ps = np.array([K.eval(layer.p) for layer in model.layers if hasattr(layer, 'p')])
sys.stdout.flush()

60000 1.6666666666666667e-13 3.3333333333333335e-05
Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 11s 231us/step - loss: 0.2263 - accuracy: 0.9229 - val_loss: 0.1301 - val_accuracy: 0.9532
Epoch 2/20
48000/48000 [==============================] - 9s 189us/step - loss: 0.0904 - accuracy: 0.9660 - val_loss: 0.0858 - val_accuracy: 0.9670
Epoch 3/20
48000/48000 [==============================] - 9s 188us/step - loss: 0.0571 - accuracy: 0.9756 - val_loss: 0.0790 - val_accuracy: 0.9702
Epoch 4/20
48000/48000 [==============================] - 9s 190us/step - loss: 0.0417 - accuracy: 0.9804 - val_loss: 0.0838 - val_accuracy: 0.9713
Epoch 5/20
48000/48000 [==============================] - 9s 188us/step - loss: 0.0321 - accuracy: 0.9823 - val_loss: 0.0670 - val_accuracy: 0.9742
Epoch 6/20
48000/48000 [==============================] - 9s 189us/step - loss: 0.0209 - accuracy: 0.9853 - val_loss: 0.0836 - val_accuracy: 0.9728
Epoch 7/2

In [9]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
concrete_dropout_1 (Concrete (None, 512)               401921    
_________________________________________________________________
concrete_dropout_2 (Concrete (None, 512)               262657    
_________________________________________________________________
concrete_dropout_3 (Concrete (None, 512)               262657    
_________________________________________________________________
concrete_dropout_4 (Concrete (None, 10)                5131      
Total params: 932,366
Trainable params: 932,366
Non-trainable params: 0
_________________________________________________________________


In [10]:
ps

array([0.03927901, 0.0347207 , 0.23016612, 0.3527136 ], dtype=float32)

In [11]:
loss, acc = model.evaluate(X_test,  y_test, verbose=0)
acc

0.9786999821662903

In [0]:
model.save_weights('model_conc_dropout.h5')

In [13]:
# This segment doesnot work as a custom layer is not seriaizable in JSON format. Changes need to be done in the layer
# itself to make compatible. It is better to just make a empty model and load weights.
# from keras.models import model_from_json
# model_json = model.to_json()
# with open("model_conc_dropout.json", "w") as json_file:
#     json_file.write(model_json)

# json_file = open('model_conc_dropout.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)

ValueError: ignored

In [14]:
N = 60000
wd = l**2. / N
dd = 2. / N
print(N,wd,dd)
inp = Input(shape=(Q,))
x = inp
x = ConcreteDropout(Dense(nb_features, activation='relu'), weight_regularizer=wd, dropout_regularizer=dd)(x)
x = ConcreteDropout(Dense(nb_features, activation='relu'), weight_regularizer=wd, dropout_regularizer=dd)(x)
x = ConcreteDropout(Dense(nb_features, activation='relu'), weight_regularizer=wd, dropout_regularizer=dd)(x)
out = ConcreteDropout(Dense(num_classes, activation='softmax'), weight_regularizer=wd, dropout_regularizer=dd)(x)
test_model = Model(inp, out)

test_model.load_weights('model_conc_dropout.h5')
assert len(test_model.weights) == len(model.weights)
for a, b in zip(test_model.weights, model.weights):
  np.testing.assert_allclose(a.numpy(), b.numpy())

60000 1.6666666666666667e-13 3.3333333333333335e-05
